In [1]:
%pip install pyspark

Defaulting to user installation because normal site-packages is not writeable
  Using cached pyspark-4.0.0.tar.gz (434.1 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached py4j-0.10.9.9-py2.py3-none-any.whl.metadata (1.3 kB)
Using cached py4j-0.10.9.9-py2.py3-none-any.whl (203 kB)
  Created wheel for pyspark: filename=pyspark-4.0.0-py2.py3-none-any.whl size=434741340 sha256=12b895121ab0436a3891019b2ab62dac251d82d464e249e6ff7d6c8862ee5b3e
  Stored in directory: c:\users\dell\appdata\local\packages\pythonsoftwarefoundation.python.3.12_qbz5n2kfra8p0\localcache\local\pip\cache\wheels\2d\77\9b\12660be70f7f447940a0caede37ae208b2e0d1c8487dce52a6
Successfully built pyspark
Note: you may need to restar


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\DELL\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date

# Inicializa a SparkSession
spark = SparkSession.builder \
    .appName("Preprocessing Dengue Data") \
    .getOrCreate()

# Caminho absoluto do arquivo CSV
csv_path = "C:/Users/DELL/Desktop/Projetos/projeto-bigdatas/data/dengue.csv"

# Leitura do CSV com cabeçalho e inferência de tipos
df = spark.read.csv(csv_path, header=True, inferSchema=True)

# Exibir as primeiras 5 linhas
df.show(5)

# Exibir o esquema dos dados
df.printSchema()

# Exemplo de limpeza: remover registros com valores nulos nas colunas principais
df_clean = df.dropna(subset=["ID_MUNICIP", "DT_NOTIFIC", "CS_SEXO", "NU_IDADE_N"])

# Converter colunas de data
df_clean = df_clean.withColumn("data_notificacao", to_date(col("DT_NOTIFIC"), "dd/MM/yyyy"))

# Filtrar apenas casos confirmados (exemplo)
df_filtrado = df_clean.filter(col("CLASSI_FIN") == 10)  # 10 = Dengue confirmado

# Exibir resultado final
df_filtrado.show(5)


+------+---------+----------+-------+------+---------+----------+----------+----------+----------+-------+--------+----------+-------+----------+-------+----------+-----+----------+----------+-------+----------+----------+-----+-------+--------+--------+------+------+----------+----------+-------+---------+----------+----------+----+---------+--------+---------+---------+-----+----------+----------+----------+----------+----------+-------+----------+----------+----------+----------+----------+------+---------+--------+----------+------+----------+--------+---------+--------+----------+----------+----+---------+---------+-------+---------+--------+----------+--------+----------+----------+--------+--------+----------+----------+---------+--------+---------+----------+----------+----------+----------+--------+-------+----------+---------+---------+----------+----------+----------+----------+----------+----------+----------+---------+--------+---------+----------+----------+-------+------

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year, month, count, desc

# Inicializa a SparkSession
spark = SparkSession.builder \
    .appName("EDA Dengue") \
    .getOrCreate()

# Lê o CSV
df = spark.read.csv("notebooks/data/dengue.csv", header=True, inferSchema=True)

# Mostra esquema
df.printSchema()

# Mostra 5 primeiras linhas
df.show(5)

# Quantidade total de registros
print("Total de registros:", df.count())

# 1. Casos por sexo
print("\nCasos por sexo:")
df.groupBy("CS_SEXO").count().show()

# 2. Casos por ano
print("\nCasos por ano (data de notificação):")
df.withColumn("ANO", year("DT_NOTIFIC")).groupBy("ANO").count().orderBy("ANO").show()

# 3. Casos por UF (estado)
print("\nCasos por estado (UF):")
df.groupBy("SG_UF_NOT").count().orderBy(desc("count")).show(10)

# 4. Casos por município
print("\nTop 10 municípios com mais casos:")
df.groupBy("ID_MUNICIP").count().orderBy(desc("count")).show(10)

# 5. Casos por evolução do caso (óbito, cura, etc.)
print("\nEvolução dos casos:")
df.groupBy("EVOLUCAO").count().orderBy(desc("count")).show()

# 6. Porcentagem de óbitos
print("\nPorcentagem de óbitos por dengue:")
total_casos = df.count()
obitos = df.filter(col("EVOLUCAO") == 2).count()  # Código 2 geralmente representa "óbito por dengue"
percentual = (obitos / total_casos) * 100
print(f"Óbitos: {obitos} de {total_casos} casos ({percentual:.2f}%)")


AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/c:/Users/DELL/Desktop/Projetos/projeto-bigdatas/notebooks/notebooks/data/dengue.csv. SQLSTATE: 42K03